In [1]:
import pandas as pd
import numpy as np

In [1]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [15]:
train = pd.read_csv('D:/tr2.csv')#, encoding='utf-8-bis')

In [11]:
test = pd.read_csv('D:/test.csv')
test['description'] = test['description'].fillna(value='')
test.shape

(508438, 17)

In [ ]:
for i in range(51):
    test['description'][i*10000:(i+1)*10000] = test['description'][i*10000:(i+1)*10000].apply(norm)
    print (i, test['description'][i*10000][:100])

In [ ]:
test.to_csv('D:/ts.csv', encoding = 'utf-8', index=False)

In [6]:
def normalize(text):
    return ' '.join([morph.parse(token)[0].normal_form for token in tknzr.tokenize(text)])
norm = np.vectorize(normalize)

In [ ]:
for i in range(151):
    train['description'][i*10000:(i+1)*10000] = train['description'][i*10000:(i+1)*10000].apply(norm)
    print (i, train['description'][i*10000][:100])

In [9]:
train.to_csv('D:/tr.csv', index=False)

In [17]:
del train

In [3]:
train.dtypes

item_id                  object
user_id                  object
region                   object
city                     object
parent_category_name     object
category_name            object
param_1                  object
param_2                  object
param_3                  object
title                    object
description              object
price                   float64
item_seq_number           int64
activation_date          object
user_type                object
image                    object
image_top_1             float64
deal_probability        float64
dtype: object

** Read **

In [2]:
train = pd.read_csv('D:/tr.csv')
#test = pd.read_csv('D:/ts.csv')

In [ ]:
train_1 = train[['region','city','parent_category_name','category_name','param_1','param_2','param_3','user_type']]
train_1 = train_1.fillna(value='')

from ce import CategoricalEncoder
c = CategoricalEncoder(handle_unknown='ignore')
%time c.fit(train_1)

train_1 = c.transform(train_1)

train_2 = train[['price','item_seq_number','image']]
train_2 = train_2.fillna(value=0)

train_3 = train[['title']]
train_2['title_len'] = train_3['title'].apply(len)
train_2 = train_2.to_sparse()

del train, train_3

In [5]:
train_1

<1503424x3684 sparse matrix of type '<class 'numpy.float64'>'
	with 12027392 stored elements in Compressed Sparse Row format>

In [7]:
train_2['image'] = train_2['image'].apply(lambda x: 0 if x is np.nan else 1)
train_2 = train_2.to_sparse()

In [8]:
from scipy.sparse import hstack
train = hstack([train_1, train_2])

In [9]:
train

<1503424x3688 sparse matrix of type '<class 'numpy.float64'>'
	with 17951080 stored elements in COOrdinate format>

In [11]:
import scipy.sparse
scipy.sparse.save_npz('D:/train12.npz', train)

In [12]:
>>> from sklearn.externals import joblib
>>> joblib.dump(c, 'categorical.pkl') 

['categorical.pkl']

In [3]:
train_1 = train[['region','city','parent_category_name','category_name','param_1','param_2','param_3','user_type']]
train_2 = train[['price','item_seq_number','image','title']]
train_3 = train['description']
y_train = train['deal_probability']

del train
train_2['image'] = train_2['image'].apply(lambda x: 0 if x is np.nan else 1)

train_1 = train_1.fillna(value='')
train_2 = train_2.fillna(value=0)
train_3 = train_3.fillna(value='')

In [4]:
from ce import CategoricalEncoder
c = CategoricalEncoder(handle_unknown='ignore')
%time c.fit(train_1)

Wall time: 14.7 s


CategoricalEncoder(categories='auto', dtype=<class 'numpy.float64'>,
          encoding='onehot', handle_unknown='ignore')

In [5]:
train_1 = c.transform(train_1)

In [6]:
train_1

<1503424x3684 sparse matrix of type '<class 'numpy.float64'>'
	with 12027392 stored elements in Compressed Sparse Row format>

In [7]:
train_2['title'] = train_3['title'].apply(len)
del train_3['title']

** TF-IDF **

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('D:/tr.csv')['description']

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train = train.fillna(value='')
_, train, = train_test_split(train, test_size=0.4, random_state=23)

In [5]:
train.shape

(601370,)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from nltk.corpus import stopwords

In [8]:
tfidf = TfidfVectorizer(stop_words = stopwords.words('russian'), ngram_range =(1,3), token_pattern='\S+', max_features=20000)
tfidf.fit(train.values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False, token_pattern='\\S+',
        tokenizer=None, use_idf=True, vocabulary=None)

In [9]:
del train

In [10]:
>>> from sklearn.externals import joblib
>>> joblib.dump(tfidf, 'tfidf.pkl') 

['tfidf.pkl']

** full train **

In [2]:
train = pd.read_csv('D:/tr.csv')['description']

In [3]:
from sklearn.externals import joblib
>>> tfidf = joblib.load('D:/tfidf.pkl') 

In [5]:
train = train.fillna(value='')

In [6]:
train = tfidf.transform(train.values)

In [7]:
import scipy.sparse
scipy.sparse.save_npz('D:/train3.npz', train)

In [8]:
del train

** full test **

In [13]:
test = pd.read_csv('D:/ts.csv')['description']
test = test.fillna(value='')
test = tfidf.transform(test.values)
scipy.sparse.save_npz('D:/test3.npz', test)

## Test

In [2]:
test = pd.read_csv('D:/test.csv')

In [4]:
>>> from sklearn.externals import joblib
>>> c = joblib.load('categorical.pkl') 

In [ ]:
test_1 = test[['region','city','parent_category_name','category_name','param_1','param_2','param_3','user_type']]
test_1 = test_1.fillna(value='')

test_1 = c.transform(test_1)

test_2 = test[['price','item_seq_number','image']]
test_2 = test_2.fillna(value=0)

test_3 = test[['title']]
test_2['title_len'] = test_3['title'].apply(len)
test_2 = test_2.to_sparse()

del test

In [8]:
del test_3

test_2['image'] = test_2['image'].apply(lambda x: 0 if x is np.nan else 1)
test_2 = test_2.to_sparse()

from scipy.sparse import hstack
test = hstack([test_1, test_2])

In [9]:
test

<508438x3688 sparse matrix of type '<class 'numpy.float64'>'
	with 6068627 stored elements in COOrdinate format>

In [11]:
import scipy.sparse
scipy.sparse.save_npz('D:/test12.npz', test)

## Classification

In [2]:
y = pd.read_csv('D:/tr.csv')['deal_probability']

In [3]:
import scipy.sparse
from scipy.sparse import hstack
train = hstack([scipy.sparse.load_npz('D:/train12.npz'), scipy.sparse.load_npz('D:/train3.npz')])

In [4]:
train

<1503424x23688 sparse matrix of type '<class 'numpy.float64'>'
	with 66269352 stored elements in COOrdinate format>

In [5]:
from sklearn.linear_model import LinearRegression

In [7]:
model = LinearRegression(normalize=True)

In [8]:
%time model.fit(train, y)

Wall time: 12min 44s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [9]:
del train

test = hstack([scipy.sparse.load_npz('D:/test12.npz'), scipy.sparse.load_npz('D:/test3.npz')])
pred = model.predict(test)

label = pd.read_csv('D:/ts.csv')['item_id']

df = pd.DataFrame()
df['item_id'] = label.values
df['deal_probability'] = pred

df.to_csv('linear.csv',index=False)

In [12]:
pred[pred < 0] = 0
pred[pred > 1] = 1

In [13]:
df = pd.DataFrame()
df['item_id'] = label.values
df['deal_probability'] = pred

df.to_csv('linear.csv',index=False)

** Ridge **